In [ ]:
import os
import shutil
import re
import markdown
import zipfile
import shutil


In [ ]:

def copy_media(md_content, md_dir, output_dir):
    # 正则表达式匹配图片引用
    image_pattern = re.compile(r'!\[.*?\]\((.*?)\)')
    images = image_pattern.findall(md_content)
    
    # 正则表达式匹配视频引用
    video_pattern = re.compile(r'<video\s+.*?src=["\'](.*?)["\'].*?>')
    videos = video_pattern.findall(md_content)
    
    media = images+videos

    for media_path in media:
        # 构造图片的绝对路径
        full_media_path = os.path.join(md_dir, media_path)
        
        if os.path.exists(full_media_path):
            # 确保目标目录存在
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            # 将图片复制到目标目录
            shutil.copy(full_media_path, output_dir)

            # 更新图片路径为 HTML 目录下的相对路径
            image_file_name = os.path.basename(media_path)
            new_media_path = os.path.join('.', image_file_name)
            md_content = md_content.replace(media_path, new_media_path)

    return md_content


In [ ]:

def convert_markdown_to_html(md_file_path, output_html_path):
    # 读取 Markdown 文件内容
    with open(md_file_path, 'r', encoding='utf-8') as md_file:
        md_content = md_file.read()

    # 获取 Markdown 文件所在目录
    md_dir = os.path.dirname(md_file_path)

    # 获取输出 HTML 所在目录
    output_dir = os.path.dirname(output_html_path)

    # 处理图片引用并复制图片
    md_content = copy_media(md_content, md_dir, output_dir)

    # 转换为 HTML
    html_content = markdown.markdown(md_content)

    # 保存为 HTML 文件
    with open(output_html_path, 'w', encoding='utf-8') as html_file:
        html_file.write(html_content)

    print(f"Markdown 文件 '{md_file_path}' 已成功转换为 HTML '{output_html_path}'")


In [ ]:
def zip_folder_and_delete_original(folder_path, output_zip_path):
    """
    Compress a folder into a .zip file and delete the original folder.

    :param folder_path: The path to the folder to compress
    :param output_zip_path: The path where the .zip file will be saved
    """
    try:
        # Create a ZIP file
        with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # Walk through the folder and add each file to the zip
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    # Create a relative path within the zip
                    relative_path = os.path.relpath(file_path, folder_path)
                    # Add the file to the zip
                    zipf.write(file_path, relative_path)
                    print(f"Added {file_path} as {relative_path}")

        print(f"Folder '{folder_path}' has been successfully zipped to '{output_zip_path}'")

        # Delete the original folder
        shutil.rmtree(folder_path)
        print(f"Original folder '{folder_path}' has been deleted.")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:

# 修改monthly_title后运行进行打包
monthly_title = 'Aug2024'
md_file_path = monthly_title+'/'+monthly_title+'.md'  # 输入的 Markdown 文件路径
output_html_path = 'temp/ '+monthly_title+'.html'  # 输出的 HTML 文件路径
output_folder_path = 'temp'
output_zip_path = monthly_title+'.zip'
convert_markdown_to_html(md_file_path, output_html_path)
zip_folder_and_delete_original(output_folder_path, output_zip_path)

